### Import modules

In [27]:
import numpy as np
import pandas as pd
import pickle

import bs4 as bs
import urllib.request
import json
from bs4 import BeautifulSoup
import requests
import re
from collections import Counter

import MySQLdb, pickle
from sqlalchemy import create_engine

### Crawling data from Cars.com

In [41]:
# Create a data frame to store crawling data
df = pd.DataFrame(columns = ['year', 'title','brand','model','miles','exterior_color','price'])

# Getting data from cars.com
for page in range(0,2):
    # Cars.com URL
    url = 'https://www.cars.com/for-sale/searchresults.action/?page='+str(page)+'&perPage=100&searchSource=PAGINATION&shippable-dealers-checkbox=true&sort=relevance&stkTypId=28881&zc=31216&localVehicles=false'  
    # Requesting data 
    sauce = urllib.request.urlopen(url).read()
    
    # Parsing data with lxml
    soup = bs.BeautifulSoup(sauce, 'lxml')
    
    # All of the data comes with listing-row__details as the class name
    specificSoup = soup.find_all('div', class_='listing-row__details')

    
    # Put the imported data in the data frame
    for div in specificSoup:
#         print(div)
        
#         # Put information about the name of the car in the name variable
        name_index = div.find('h2', {'class' :'listing-row__title'}).text
#         print(name_index)
        name = name_index.split("\n")[1]
#         print(name)

#         # Get only the car's year out of the car information in the name variable
        year_index = re.findall('[0-9]{4}',name)[0:1]
        year = year_index[0]
#         print(year)

        # Get only the car's title out of the car information in the name variable
        title_index = name.split(" ")[29:34]
        title = " ".join(title_index)
#         print(title)

        # Get the car's brand out of the car information in the title variable
        brand = title.split(" ")[4]
#         print(brand)
        # If there hasn't model information, brand will be as a model
        try:
            model = title.split(" ")[4]
        except:
            model = brand

        # The miles of the car 
        mile_index = div.find('span', {'class' : 'listing-row__mileage'}).text
#         print(mile_index)
        miles = mile_index.split("\n")[1].strip()
        regex = re.compile("\d+")
        miles = regex.findall(miles) 
        miles = ''.join(miles)
        miles = miles.strip()
        
        if miles == "":
            miles = 0
        else:
            miles

        # Exterior color
        exterior_color = div.find('ul', {'class' : 'listing-row__meta'}).text
        try:
            exterior_color = re.sub('\n', ' ',exterior_color).split(" ")[45]
        except:
            exterior_color = 'black'
        
        
        # Interior color
#         interior_color = div.find('ul', {'class' : 'listing-row__meta'})
#         print(interior_color)
#         try:
#             interior_color = list(interior_color)[3].text.split(" ")[3:5]
#         except:
#             interior_color = "black"
#         interior_color = " ".join(interior_color)
#         interior_color = re.sub('/' , ' ' ,interior_color)

#         # Transmission type
#         transmission = div.find('ul', {'class' : 'listing-row__meta'})
#         try:
#             transmission = list(transmission)[5].text.split(" ")[2]
#         except:
#             transmission = "6-speed"
        
#         # Drivetrain type
#         drivetrain = div.find('ul', {'class' : 'listing-row__meta'})
#         try:
#             drivetrain = list(drivetrain)[7].text.split(" ")[2:5]
#         except: 
#             drivetrain = 'fwd'
#         drivetrain = " ".join(drivetrain).lower()
#         if drivetrain == 'rear wheel drive':
#             drivetrain = 'rwd'
#         elif drivetrain == 'front wheel drive':
#             drivetrain = 'fwd'
#         elif drivetrain == 'all wheel drive':
#             drivetrain = '4wd'

#         # Number of star
#         if div.find('div',{'class' : 'dealer-rating-stars'}) == None:
#             star = 0
#         else:
#             star_index =div.find('div',{'class' : 'dealer-rating-stars'}).text
#             star = star_index.split(" ")[36]
#             regex = re.compile("\d")
#             star = regex.findall(star)[0] 

#         # Number of review
#         if div.find('span',{'class' : 'listing-row__review-number'}) == None:
#             review_no = 0
#         else:
#             review_index =div.find('span',{'class' : 'listing-row__review-number'}).text
#             review_no = re.sub('\n', '',review_index.split(" ")[1])
#             try:
#                 review_no = re.sub('[()]', '',review_no)
#             except:
#                 review_no = review_no

        # Car price
        if div.find('span', {'class' : 'listing-row__price'}) == None:
            price = 0
        else:
            price_index = div.find('span', {'class' : 'listing-row__price'}).text
            price = price_index.split("\n")[1]
            regex = re.compile("\d")
            price = ''.join(regex.findall(price))


#         # Entering crawled data into a data frame
        data = { 
                'year' : year,
                'title' : title.lower(),
                'brand': brand.lower(),
                'model': model.lower(),
                'miles' : miles,
                'exterior_color' : exterior_color.lower(),
#                 'interior_color' : interior_color.lower(),
#                 'transmission' : transmission.lower(),
#                 'drivetrain' : drivetrain.lower(),
#                 'star': star,
#                 'review_no' : review_no,
                'price': price,
                    }


        df.loc[len(df)] = data
# # Data Preprocessing   
# df['transmission'] = df['transmission'].apply(lambda x: '6-speed' if x == 'automatic' or x == '6' else x)
# df['transmission'] = df['transmission'].apply(lambda x: '8-speed' if x == '8' else x)
# df['transmission'] = df['transmission'].apply(lambda x: '5-speed' if x == '5' else x)   
# df['transmission'] = df['transmission'].apply(lambda x: '5-speed' if x == '5' else x)   
# df['transmission'] = df['transmission'].apply(lambda x: 'x-speed' if x != '1-speed' and x != '2-speed' and x != '3-speed' and \
#                                               x != '4-speed' and x != '5-speed' and x != '6-speed' and x != '7-speed' and x != '7-speed' and\
#                                               x != '8-speed' and x != '9-speed' and x != '10-speed' \
#                                               else x)
# df['drivetrain'] = df['drivetrain'].apply(lambda x: '4wd' if x == 'four wheel drive' or x == '4wd' or x=='4x4'or x=='awd'else x)
# df['drivetrain'] = df['drivetrain'].apply(lambda x: 'fwd' if x == '2wd' or x=='f w d' else x)
# df['drivetrain'] = df['drivetrain'].apply(lambda x: '4wd' if x != 'fwd' and  x!='rwd' and x!='4wd' else x)

df

year              title      brand      model   miles exterior_color  \
0    2012     2012 chevrolet  chevrolet  chevrolet   95802          brown   
1    2018      2018 chrysler   chrysler   chrysler   12522          white   
2    2016         2016 lexus      lexus      lexus   28798          white   
3    2011        2011 nissan     nissan     nissan  100332           gray   
4    2015          2015 ford       ford       ford   20713          white   
5    2019        2019 toyota     toyota     toyota   12693          white   
6    2019         2019 honda      honda      honda     232          white   
7    2017     2017 chevrolet  chevrolet  chevrolet   25331          black   
8    2013          2013 ford       ford       ford   74489           gray   
9    2015        2015 nissan     nissan     nissan   51983          black   
10   2017          2017 ford       ford       ford   44411         silver   
11   2016      2016 cadillac   cadillac   cadillac   44983          white   
12   2016           2016 gmc        gmc        gmc   44268           gray   
13   2007     2007 chevrolet  chevrolet  chevrolet   13464            red   
14   2013      2013 infiniti   infiniti   infiniti   79546           gray   
15   2018          2018 audi       audi       audi   23671          white   
16   2018      2018 chrysler   chrysler   chrysler   38117          white   
17   2016          2016 ford       ford       ford   33003         silver   
18   2015     2015 chevrolet  chevrolet  chevrolet   65661           gray   
19   2019         2019 dodge      dodge      dodge    4641            red   
20   2013           2013 bmw        bmw        bmw   46334           gray   
21   2018         2018 dodge      dodge      dodge   11938            red   
22   2019        2019 nissan     nissan     nissan   15783         silver   
23   2011      2011 cadillac   cadillac   cadillac   49490          black   
24   2014         2014 dodge      dodge      dodge   85483           blue   
25   2017          2017 jeep       jeep       jeep    8449         silver   
26   2013     2013 chevrolet  chevrolet  chevrolet   94957         silver   
27   2013        2013 toyota     toyota     toyota   86214            red   
28   2013          2013 ford       ford       ford   24181         silver   
29   2015           2015 gmc        gmc        gmc  105444          black   
..    ...                ...        ...        ...     ...            ...   
170  2017       2017 hyundai    hyundai    hyundai   50535           gray   
171  2018          2018 ford       ford       ford   27306         silver   
172  2013     2013 chevrolet  chevrolet  chevrolet  119909          green   
173  2019        2019 nissan     nissan     nissan    1250          black   
174  2018          2018 jeep       jeep       jeep     110           blue   
175  2015      2015 cadillac   cadillac   cadillac   42026         silver   
176  2011          2011 ford       ford       ford  119472          black   
177  2015          2015 ford       ford       ford   89209          white   
178  2011     2011 chevrolet  chevrolet  chevrolet  171298          black   
179  2019        2019 nissan     nissan     nissan   11038          black   
180  2013         2013 buick      buick      buick   40865          white   
181  2019        2019 nissan     nissan     nissan   11515          green   
182  2015        2015 toyota     toyota     toyota   40383           blue   
183  2014     2014 chevrolet  chevrolet  chevrolet   80590          white   
184  2012           2012 gmc        gmc        gmc   67659         silver   
185  2018         2018 honda      honda      honda      31           gray   
186  2012        2012 nissan     nissan     nissan   65170           gray   
187  2012          2012 audi       audi       audi   52408           blue   
188  2013          2013 jeep       jeep       jeep   82096          white   
189  2015         2015 acura      acura      acura   49268         silver   
1

In [42]:
# Exclude prices and non-branded data
df = df[df["price"] != ""]
df = df[df["brand"] != ""]

In [43]:
# Transforming data for data processing
df["year"] = df["year"].astype('int')
df["miles"] = df["miles"].astype('int')
# df["photos"] = df["photos"].astype('int')
# df["video"] = df["video"].astype('int')
# df["star"] = df["star"].astype('int')
# df["review_no"] = df["review_no"].astype('int')
df["price"] = df["price"].astype('int')

### Put Crawled data into Database (MySQL) at Amazon web service cloud

In [44]:
# Load pickle file as database password
pw = pickle.load(open('./Flask/models/pw.plk','rb'))

In [45]:
# Saving data to the database(MySQL at Amazon Web Service Cloud )
engine = create_engine("mysql+mysqldb://root:" + pw + "@localhost/car", pool_pre_ping=True)
# df.to_sql(name="car", con=engine, if_exists='replace')

In [46]:
 df.to_sql(name="car", con=engine, if_exists='replace')